In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import argparse
import datetime
import math

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

# The above could be sent to an independent module
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind

import statsmodels.tsa.stattools as smts
import statsmodels.api as sm

import datetime as dt

** TODO **

- ur first step is to read all data from NYSE as dataframes(narrow down to NYSE for now)
- look at the starting date of all dataframes, pick the max, and then segment all dataframe to make sure they start on the same day (u can refer to notebook 4c "generate_pairs_df" to see how this can be done)
- iterate through all NC2 pair, and calculate their sum of squared difference for each pair, then pick K smallest pairs, save them somewhere (YOU DONT HAVE TO WRITE THE CODE FOR THIS, ITS ALRDY WRITTEN BY GORDON IN **pair_selectory.py**, just figure out how to pass the price data of all N stocks in as the right format, if i rmb correctly the format is, each column is the price of 1 stock, column name is the stock name, so u need to aggregate close price of all N stocks into 1 dataframe)
- for each pair, backtest and save result metrics (initial portfolio val, final porfolio val, number of timeouts) to a pandas (dont just print lol)
- make sure to save the name of the pair also lol (like 'GOOG-GOOGL')
- write the pandas to a csv 
- to read all csv files from a directory, u need to import glob and os (check sample code in notebook 4a to see how i read all csv files from one directory as a list)


In [ ]:
class SinglePair(bt.Strategy):
    params = dict (
        period=84,
    )

    def log(self, txt, dt=None):        
        dt = dt or self.data.datetime[0]
        dt = bt.num2date(dt)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [bt.Order.Submitted, bt.Order.Accepted]:
            return  # Await further notifications

        if order.status == order.Completed:
            if order.isbuy():
                buytxt = 'BUY COMPLETE, %.2f' % order.executed.price
                self.log(buytxt, order.executed.dt)
                self.incur_commission(order.executed.price, order.executed.size)
            else:
                selltxt = 'SELL COMPLETE, %.2f' % order.executed.price
                self.log(selltxt, order.executed.dt)
                self.incur_commission(order.executed.price, order.executed.size)

        elif order.status in [order.Expired, order.Canceled, order.Margin]:
            self.log('%s ,' % order.Status[order.status])
            pass  # Simply log

        # Allow new orders
        self.orderid = None

    def __init__(self):
        # To control operation entries
        self.orderid = None
        self.qty1 = 0
        self.qty2 = 0
        self.period = self.p.period
        self.upper_limit = 2.0
        self.lower_limit = -2.0
        self.up_medium = 0.5
        self.low_medium = -0.5
        self.status = 0
        self.exposure = 200000
        
        self.data0_log = Log(self.data0.close)
        self.data1_log = Log(self.data1.close)
        
        self.zscore = None
        self.adf_pvalue = None
        self.intercept = None
        self.slope = None
        self.resid_mean = None
        self.resid_std = None
        self.spread = None
        self.zscore = None
        
        self.open_pos_duration = 0
        self.n_timeouts = 0

    def next(self):
        print("--------------------------------------------------------------------")
        print("Current cash: %.2f" % self.broker.get_cash())
        print("Portfolio value: %.2f" % self.broker.getvalue())
        
        if min(len(self.data0_log), len(self.data1_log)) < self.period:
            return
        
        if self.orderid:
            return  # if an order is active, no new orders are allowed
        
        ##################################################################################################
        # COMPUTE UPPER AND LOWER LIMITS                                                                 #
        ##################################################################################################
        Y = pd.Series(self.data0.get(size=self.period)[0:-1])
        X = pd.Series(self.data1.get(size=self.period)[0:-1])
        
        self.spread = (self.data0[0] - self.data1[0])
        
        if self.status == 0:
            self.spread_mean = (Y - X).mean()
            self.spread_std = (Y - X).std()

            self.upper_limit = self.spread_mean + 2 * self.spread_std
            self.lower_limit = self.spread_mean - 2 * self.spread_std
            self.up_medium = self.spread_mean + 0.5 * self.spread_std
            self.low_medium = self.spread_mean - 0.5 * self.spread_std
        
        ##################################################################################################
        # LOGGING                                                                                        #
        ##################################################################################################
        print("--------------------------------------------------------------------")
        
        print('Strategy  len:', len(self))
        print('Data0 len:', len(self.data0))
        print('Data1 len:', len(self.data1))
        
        print('Data0 dt:', dt.datetime.strftime(data0.datetime.datetime(),'%Y-%m-%d %H:%M:%S'))
        print('Data1 dt:', dt.datetime.strftime(data1.datetime.datetime(),'%Y-%m-%d %H:%M:%S'))

        print('Data0 dt:', dt.datetime.strftime(data0.datetime.datetime(),'%Y-%m-%d %H:%M:%S'))
        print('Data1 dt:', dt.datetime.strftime(data1.datetime.datetime(),'%Y-%m-%d %H:%M:%S'))
        
        print('status is', self.status)
    
        ##################################################################################################
        # STRATEGY LOGIC                                                                                 #
        ##################################################################################################
        if self.status == 0:
            # "NO position" status
            self.open_pos_duration = 0
            
            if self.spread > self.upper_limit:
                self.short_spread()
            elif self.spread < self.lower_limit:
                self.long_spread()
     
        elif self.status == 1:
            # "SHORT the spread" status
            self.open_pos_duration += 1
            
            if self.spread < self.lower_limit:
                self.long_spread()
            elif self.spread < self.up_medium:
                self.exit_spread()
                self.status = 0
        
        elif self.status == 2:
            # "LONG the spread" status
            self.open_pos_duration += 1
            
            if self.spread > self.upper_limit:
                self.short_spread()
            elif self.spread > self.low_medium:
                self.exit_spread()
                self.status = 0
                
        # TIMEOUT condition
        if self.open_pos_duration > 84:
            self.exit_spread()
            self.n_timeouts += 1
            self.status = 0
    
    def short_spread(self):
        value = 0.5 * self.exposure  
        x = int(value / (self.data0.close))  
        y = int(value / (self.data1.close))  
        
        print('x + self.qty1 is', x + self.qty1)
        print('y + self.qty2 is', y + self.qty2)

        # Placing the order
        self.log('SELL CREATE %s, price = %.2f, qty = %d' % ("STOCK 1", self.data0.close[0], x + self.qty1))
        self.sell(data=self.data0, size=(x + self.qty1))  # Place an order for buying y + qty2 shares
        self.log('BUY CREATE %s, price = %.2f, qty = %d' % ("STOCK 2", self.data1.close[0], y + self.qty2))
        self.buy(data=self.data1, size=(y + self.qty2))  # Place an order for selling x + qty1 shares

        # Updating the counters with new value
        self.qty1 = x  
        self.qty2 = y  
        
        # update flags
        self.status = 1
        self.open_pos_duration = 0
    
    def long_spread(self):
        # Calculating the number of shares for each stock
        value = 0.5 * self.exposure 
        x = int(value / (self.data0.close)) 
        y = int(value / (self.data1.close)) 
        
        print('x + self.qty1 is', x + self.qty1)
        print('y + self.qty2 is', y + self.qty2)

        # Place the order
        self.log('BUY CREATE %s, price = %.2f, qty = %d' % ("STOCK 1", self.data0.close[0], x + self.qty1))
        self.buy(data=self.data0, size=(x + self.qty1))  # Place an order for buying x + qty1 shares
        self.log('SELL CREATE %s, price = %.2f, qty = %d' % ("STOCK 2", self.data1.close[0], y + self.qty2))
        self.sell(data=self.data1, size=(y + self.qty2))  # Place an order for selling y + qty2 shares

        # Updating the counters with new value
        self.qty1 = x 
        self.qty2 = y 
        
        # update flags
        self.status = 2  
        self.open_pos_duration = 0
    
    def exit_spread(self):
        # Exit position
        self.log('CLOSE position %s, price = %.2f' % ("STOCK 1", self.data0.close[0]))
        self.close(self.data0)
        self.log('CLOSE position %s, price = %.2f' % ("STOCK 2", self.data1.close[0]))
        self.close(self.data1)
        
        # update counters
        self.qty1 = 0
        self.qty2 = 0
        
        # update flags
        self.status = 0
        self.open_pos_duration = 0
        
    def incur_commission(self, price, qty):
        qty = abs(qty)
        commission = min(max(1, 0.005*qty), 0.01*price*qty)
        print('Commission of ' + str(commission) + 'USD incurred.')
        self.broker.add_cash(-1*commission)
    
    def stop(self):
        print('==================================================')
        print('Starting Value: %.2f' % self.broker.startingcash)
        print('Ending   Value: %.2f' % self.broker.getvalue())
        print('Number of timeouts: %.2f' % self.n_timeouts)
        print('==================================================')

- This is a class for backtesting (dist method) on a single pair
- look at notebook 4b to see how to initialize a cerebro object, pass 2 data feeds in and run backtest